In [1]:
import emblaze
from emblaze.utils import Field, ProjectionTechnique
import numpy as np
import pandas as pd

from sklearn.manifold import TSNE
import numpy as np
from matplotlib import pyplot as plt
from data.visamm_dataset import make_mm_dataset
from data.image_folder import make_dataset
import os
from PIL import Image
from tqdm import tqdm

# Load Embeddings

In [2]:

np.random.seed(42)

IS_MM = False


# dataroot = '../pre-processing/subset_classes_single'
dataroot = '../pre-processing/subset_classes'
# dataroot = '../pre-processing/mmdata'

# resultsroot= './saved_results/five-classes/bear/latent_code/bear_256x512/test_latest/images'
# resultsroot= './results/debug/latent_code/debug/test_latest/images'
resultsroot= './results/visamm-gpt-five/latent_code/visamm-gpt-five/test_latest/images'

outpath = './tsne-five.png'

# resultsroot= './saved_results/single-class/bear/latent_code/bear_256x512/test_latest/images'
# outpath = './tsne-single.png'

SUBSETS = {
    1: { 
        'bicycling': 1,    
        'drawing': 2,
        'driving': 3,
        'running': 4,
        'surfing': 5
    }
}

if IS_MM:
    paths = sorted(make_mm_dataset(dataroot, 1), key=lambda d: d["im_path"])
    paths = [p["im_path"] for p in paths]
else:
    paths = sorted(make_dataset(dataroot))

# embeddings_to_plot = 500

name2idx = {
    'bicycling': 1,    
    'drawing': 2,
    'driving': 3,
    'running': 4,
    'surfing': 5
}

class_indices = []
class_names = []
image_embeddings = []


for idx, path in tqdm(enumerate(paths)):
    classname = path.split('/')[-3]
    class_indices.append(name2idx[classname])
    class_names.append(classname)
print("classes loaded")

result_fnames = os.listdir(resultsroot)
result_fnames.sort(key=lambda p: int(p.split("_")[0]))
for idx, fname in tqdm(enumerate(result_fnames)):
    fpath = os.path.join(resultsroot, fname)
    embedding = np.load(fpath)
    image_embeddings.append(embedding.flatten())
print("embeddings loaded")

44112it [00:00, 479326.68it/s]


classes loaded


44112it [00:30, 1423.91it/s]

embeddings loaded


# Load Images

In [3]:
# 0-300, 5000-5500
START = 0
END = 300

# START = 0
# END = 300

images = []
for idx, path in tqdm(enumerate(paths[START:END])):
    with Image.open(path) as img:
        images.append(np.asarray(img))
    
print("images loaded")

300it [00:01, 268.21it/s]

images loaded


# Get Visualization

In [4]:
images_arr = np.array(images)
# embeddings_arr = np.array([e.flatten() for e in image_embeddings])
embeddings_arr = np.array([e.flatten() for e in image_embeddings[START:END]])                           
classes_arr = np.array(class_indices[START:END])

# Computing neighbors

emb = emblaze.Embedding({Field.POSITION: embeddings_arr, Field.COLOR: classes_arr})
# print("Computing neighbors"）
# emb.compute_neighbors(metric='cosine')
      
print("Getting projections")
variants = emblaze.EmbeddingSet([
    emb.project(method=ProjectionTechnique.UMAP) for _ in range(10)
])

print("Computing neighbors")
variants.compute_neighbors(metric='euclidean')

print("Producing visualization")
thumbnails = emblaze.ImageThumbnails(images)
w = emblaze.Viewer(embeddings=variants, thumbnails=thumbnails)
w

<ipython-input-4-c781834aceaf>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  images_arr = np.array(images)


Getting projections
Computing neighbors
Producing visualization


Viewer(colorScheme='tableau', data={'data': [{'_format': 'compressed', '_idtype': 'u2', '_length': 300, 'ids':…

In [ ]:
variants

In [ ]:
w

In [ ]:


# data = list(zip(class_names, image_embeddings, class_indices))
# np.random.shuffle(data)
# data = data[:embeddings_to_plot]
# class_names = [p[0] for p in data]
# image_embeddings = [p[1] for p in data]
# class_indices = [p[2] for p in data]


print("doing tsne...")
tsne_embedded = TSNE(n_components=2, n_jobs=32,
                     learning_rate="auto", init="pca").fit_transform(np.array([e.flatten() for e in image_embeddings]))
print("done")

fig = plt.figure(1, (17., 15.))
# plt.scatter(tsne_embedded[:, 0], tsne_embedded[:, 1], alpha=1.0, s=2,
#             c=[v for v in class_indices], cmap="plasma")
scatter = plt.scatter(tsne_embedded[:, 0], tsne_embedded[:, 1], alpha=1.0, s=2,
            c=[v for v in class_indices], cmap="plasma", label=list(name2idx.keys()))
plt.legend(loc='upper right', handles=scatter.legend_elements()[0], labels=name2idx.keys())
# # cb = plt.colorbar()
# cb.set_label('Category')
# plt.savefig(outpath)
plt.show()

In [ ]:
import clip